# Which counties in Germany have alot of vehicles registered per inhabitant?

In this project we examined the correlation between the amount of inhabitants and the amount of registered vehicles for german counties. This study aims to identify counties with a large number of vehicles per inhabitant. By identifying such counties the german government could start improving public transport in this counties and therefore offer more people ecofriendly transportation.

For that purpuse we used data from Mobilithek about the vehicles per county (https://mobilithek.info/offers/-8395690728355365851) and an additional data set about the inhabitants per county (https://downloads.suche-postleitzahl.org/v2/public/plz_einwohner.csv).

## Install dependencies

In [55]:
%pip install pandas
%pip install 'SQLAlchemy==1.4.46'
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [56]:
import pandas as pd
import sqlalchemy as sa
import os
import matplotlib.pyplot as plt

#df = read_sql('sqlite:///data.sqlite')

## Load the data from the Database into a DataFrame

### Query the data needed for the data analysis

Now I could run a query on the data as they are stored in the SQLite-Database. To answer my question I selected everything what has been imported from the county and vehicle data.
After that the query gets executed and stored into a dataframe.

In [79]:
query = sa.text("""
SELECT c.note AS County, c.einwohner AS Inhabitants, v.Pkw AS "Registered Vehicles", v.Ackerschlepper AS Ackerschlepper
FROM county AS c
JOIN vehicle AS v ON c.note = v.county
""")
current_directory = os.getcwd()
print("Current Directory:", current_directory)
database_path = os.path.join(current_directory, "data", "data.sqlite")

engine = sa.create_engine(f'sqlite:///{database_path}')
conn = engine.connect() # Execute the query and fetch the data
result = conn.execute(query)
data = result.fetchall()

#df = pd.DataFrame(data, columns=["County", "Inhabitants", "Registered Vehicles", "Ackerschlepper"]) # Create the DataFrame

Current Directory: /workspaces/2023-amse-template_LSc


OperationalError: (sqlite3.OperationalError) no such table: county
[SQL: 
SELECT c.note AS County, c.einwohner AS Inhabitants, v.Pkw AS "Registered Vehicles", v.Ackerschlepper AS Ackerschlepper
FROM county AS c
JOIN vehicle AS v ON c.note = v.county
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

### Set up the Data for the exploratory analysis and initiate important variables

In [ ]:
filtered_df_pkw = df[df["Registered Vehicles"].notnull() & (df["Registered Vehicles"] != "-")] # Filter out non-integers and NULLs, we do this in this step as otherwise we would need to store a '0' in our DB 
filtered_df_ackerschlepper = df[df["Ackerschlepper"].notnull() & (df["Ackerschlepper"] != "-")] # or use dropna() which would drop a column even if it would only lack one value

max_registered_vehicles = filtered_df_pkw["Registered Vehicles"].astype(int).max() # Convert max_registered_vehicles and max_inhabitants to integers and get the maximum value
max_inhabitants = filtered_df_pkw["Inhabitants"].astype(int).max()

num_ticks = 10 # Set the desired number of tick marks

x_ticks_pkw = list(range(0, max_registered_vehicles + 1, max_registered_vehicles // num_ticks)) # Define the tick positions for the x-axis
x_ticks_ackerschlepper = list(range(0, filtered_df_ackerschlepper["Ackerschlepper"].astype(int).max() + 1, filtered_df_ackerschlepper["Ackerschlepper"].astype(int).max() // num_ticks))


max_inhabitants = filtered_df_pkw["Inhabitants"].astype(int).max() # Again convert inhabitants into integers and then get the maximum

y_ticks = list(range(0, max_inhabitants + 1, max_inhabitants // num_ticks))

### Data exploration
Print some basic information about the data. Your data exploration would continue here.

In [ ]:
filtered_df_pkw.info()
filtered_df_ackerschlepper.info()

### Set up all parameters for the plot

In [ ]:
# Plot the counties inhabitants and their amount of registered vehicles
plt.figure(figsize=(18, 6))

#### Set up plot for PKW

In [ ]:
plt.subplot(1, 2, 1)
plt.scatter(filtered_df_pkw["Registered Vehicles"].astype(int), filtered_df_pkw["Inhabitants"].astype(int))
plt.xlabel("Registered Vehicles (Pkw)")
plt.ylabel("Inhabitants")
plt.title("County Analysis - Pkw")
plt.xticks(x_ticks_pkw)
plt.yticks(y_ticks)
plt.ticklabel_format(style="plain")
total_points_pkw = len(filtered_df_pkw)
plt.text(0.95, 0.95, f"Total Counties: {total_points_pkw}", transform=plt.gca().transAxes, ha='right', va='top')
highest_ratio_county_pkw = filtered_df_pkw.loc[(filtered_df_pkw["Registered Vehicles"].astype(int) / filtered_df_pkw["Inhabitants"].astype(int)) == (filtered_df_pkw["Registered Vehicles"].astype(int) / filtered_df_pkw["Inhabitants"].astype(int)).max(), "County"].iloc[0]
highest_ratio_value_pkw = (filtered_df_pkw["Registered Vehicles"].astype(int) / filtered_df_pkw["Inhabitants"].astype(int)).max()
lowest_ratio_county_pkw = filtered_df_pkw.loc[(filtered_df_pkw["Registered Vehicles"].astype(int) / filtered_df_pkw["Inhabitants"].astype(int)) == (filtered_df_pkw["Registered Vehicles"].astype(int) / filtered_df_pkw["Inhabitants"].astype(int)).min(), "County"].iloc[0]
lowest_ratio_value_pkw = (filtered_df_pkw["Registered Vehicles"].astype(int) / filtered_df_pkw["Inhabitants"].astype(int)).min()
plt.text(0.95, 0.90, f"Most Pkw per Inhabitant: {highest_ratio_county_pkw} ({highest_ratio_value_pkw:.2f})", transform=plt.gca().transAxes, ha='right', va='top')
plt.text(0.95, 0.85, f"Lowest Pkw per Inhabitant: {lowest_ratio_county_pkw} ({lowest_ratio_value_pkw:.2f})", transform=plt.gca().transAxes, ha='right', va='top')
filtered_df_pkw.to_sql("result_pkw", engine, if_exists="replace", index=False)

#### Set up plot for "Ackerschlepper"/Tractors

In [ ]:
plt.subplot(1, 2, 2)
plt.scatter(filtered_df_ackerschlepper["Ackerschlepper"].astype(int), filtered_df_ackerschlepper["Inhabitants"].astype(int))
plt.xlabel("Tractors")
plt.ylabel("Inhabitants")
plt.title("County Analysis - Ackerschlepper")
plt.xticks(x_ticks_ackerschlepper)
plt.yticks(y_ticks)
plt.ticklabel_format(style="plain")
total_points_ackerschlepper = len(filtered_df_ackerschlepper)
plt.text(0.95, 0.95, f"Total Counties: {total_points_ackerschlepper}", transform=plt.gca().transAxes, ha='right', va='top')
highest_ratio_county_ackerschlepper = filtered_df_ackerschlepper.loc[(filtered_df_ackerschlepper["Ackerschlepper"].astype(int) / filtered_df_ackerschlepper["Inhabitants"].astype(int)) == (filtered_df_ackerschlepper["Ackerschlepper"].astype(int) / filtered_df_ackerschlepper["Inhabitants"].astype(int)).max(), "County"].iloc[0]
highest_ratio_value_ackerschlepper = (filtered_df_ackerschlepper["Ackerschlepper"].astype(int) / filtered_df_ackerschlepper["Inhabitants"].astype(int)).max()
lowest_ratio_county_ackerschlepper = filtered_df_ackerschlepper.loc[(filtered_df_ackerschlepper["Ackerschlepper"].astype(int) / filtered_df_ackerschlepper["Inhabitants"].astype(int)) == (filtered_df_ackerschlepper["Ackerschlepper"].astype(int) / filtered_df_ackerschlepper["Inhabitants"].astype(int)).min(), "County"].iloc[0]
lowest_ratio_value_ackerschlepper = (filtered_df_ackerschlepper["Ackerschlepper"].astype(int) / filtered_df_ackerschlepper["Inhabitants"].astype(int)).min()
plt.text(0.95, 0.90, f"Most Tractors per Inhabitant: {highest_ratio_county_ackerschlepper} ({highest_ratio_value_ackerschlepper:.2f})", transform=plt.gca().transAxes, ha='right', va='top')
plt.text(0.95, 0.85, f"Lowest Tractors per Inhabitant: {lowest_ratio_county_ackerschlepper} ({lowest_ratio_value_ackerschlepper:.2f})", transform=plt.gca().transAxes, ha='right', va='top')
filtered_df_ackerschlepper.to_sql("result_ackerschlepper", engine, if_exists="replace", index=False)

### Show the Plots

In [ ]:
plt.tight_layout()
plt.show()